# Diversas fuentes de datos

![rdb](https://cdn.pixabay.com/photo/2016/12/09/18/30/database-schema-1895779_960_720.png)

## Datos relacionales

Los tipos de datos más simples que hemos visto consisten de una sola tabla con algunas variables (columnas) y algunos registros (filas). Este tipo de datos es fácil de analizar, y muchas veces podemos reducir nuestros datos a una única tabla antes de empezar a correr algoritmos de aprendizaje de máquinas sobre ellos.

Sin embargo, los datos en el mundo real no necesariamente son tan "bonitos". La mayoría de datos reales que nos encontramos son complejos y desordenados, y no son fáciles de organizar en una sola tabla sin antes hacer un buen trabajo en su procesamiento.

Adicionalmente, muchas veces podemos reducir el costo de guardar los datos en memoria distribuyendo los datos en varias tablas con relaciones definidas, en lugar de una sola tabla que concentre toda la información.

El día de hoy vamos a revisar un poco como combinar datos de diferentes fuentes, y cómo podemos generar características bastante útiles. 

Como ejemplo tomaremos datos de las 10 compañías top en el índice [Fortune Flobal 500](https://en.wikipedia.org/wiki/Fortune_Global_500). Para trabajarlas, usaremos la función `read_html` de pandas, la cual nos permitirá ingerir los datos directamente desde la página.

In [ ]:
# Importar librerías


In [ ]:
# Cargar datos en memoria usando pd.read_html


### ¿Qué es lo que hay detrás de la función `pd.read_html`?

Los pasos se pueden detallar tanto como se quiera, pero escencialmente son #:

1. Hacer un **GET request** a la página web (usando la librería [requests](https://docs.python-requests.org/en/master/)):

In [ ]:
# Importar librería requests


In [ ]:
# Hacer un get request a la página


¿Qué obtenemos con este request?

In [ ]:
# Atributo text


Inspeccionar página ...

Entonces, obtenemos todos los datos de la página. Lo "único" que nos hace falta es:

2. Llevar estos datos a un formato adecuado usando un **HTML parser** (usamos [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)):

In [ ]:
# Importar bs4.BeautifulSoup


In [ ]:
# Instanciar un objeto tipo BeautifulSoup con los contenidos del request


[Entrada de stackoverflow donde se discuten los diferentes parsers](https://stackoverflow.com/questions/25714417/beautiful-soup-and-table-scraping-lxml-vs-html-parser)

¿Qué contiene nuestra "sopa"?

Podemos hacer búsqueda de diferentes objetos:

In [ ]:
# Título


In [ ]:
# Tablas


Observamos que la tabla la podríamos "parsear" usando la clase `str`. Acá podríamos hacer uso del parser que nos provee pandas:

De este modo podemos obtener información relevante de páginas web públicas.

Hay mucho más en cuanto al tema de scraping de páginas web. 

- Si para obtener información de una página debes navegar en ella, hacer clicks en botones o cosas por el estilo, hay otra librería que nos puede ayudar a automatizar estas tareas. Su nombre es [Selenium](https://selenium-python.readthedocs.io/).

- Por otra parte, cuando una página web no quiere que sus contenidos sean obtenidos de manera masiva y repetitiva, normalmente incluyen sistemas "antibots":

![antibots](https://miro.medium.com/max/1400/1*4NhFKMxr-qXodjYpxtiE0w.gif)

- Otra práctica común, es que limiten los requests cuando identifican que se hacen con la misma dirección ip.

Volviendo a nuestros datos:

In [ ]:
# Datos de fortune 500


Una pregunta que quisieramos resolver es, ¿Cuál es el ingreso promedio por empleado?

Podemos buscar estos datos en Wikipedia también. Yo ya los "scrapeé" manualmente por ustedes para que los usemos en la clase:

In [17]:
other_data = [
    {"name": "Walmart",
     "employees": 2200000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 1566000,
     "year founded": 2002
    },
    {"name": "China National Petroleum Corporation",
     "employees": 460724,
     "year founded": 1988
    },
    {"name": "Berkshire Hathaway Inc.",
     "employees": 360000,
     "founded": 1839
    },
    {"name": "BP plc",
     "employees": 70100,
     "year founded": 1909
    },
    {"name": "China Petrochemical Corporation",
     "employees": 582648,
     "year founded": 1998
     },
    {"name": "Royal Dutch Shell",
     "employees": 86000,
     "year founded": 1907
    },
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937
    },
    {"name": "Saudi Aramco",
     "employees": 66800,
     "year founded": 1933
    },
    {"name": "Apple Inc.",
     "employees": 147000,
     "founded": 1976
    },
    {"name": "Volkswagen AG",
     "employees": 307342,
     "year founded": 1937
    },
    {"name": "Amazon.com, Inc.",
     "employees":1298000,
     "year founded": 1994
    }
]

Pensaríamos que podría ser tan fácil como hacer un merge entre ambas tablas sobre el nombre de las columnas. Sin embargo, es fácil notar que no todos los nombres coinciden.

In [ ]:
# Diccionario de mapeo entre nombres


In [ ]:
# Hacer un map de los nombres en el dataframe inicial


In [ ]:
# Convertir el diccionario anterior a dataframe


In [ ]:
# Hacer el merge


In [ ]:
# Responder la pregunta


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>